## Python imports

In [1]:
import os
import sys
import requests
from bs4 import BeautifulSoup
from jinja2 import Template

### _Load Django into Jupyter_

In [2]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

## Show Django Admin Page Title

In [3]:
url = 'http://bookstore:8000/admin'

def page_title(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup.title.string

page_title(url)

'Log in | Django site admin'

## Add Application

In [4]:
applist = "INSTALLED_APPS = [\n\
    'django.contrib.admin',\n\
    'django.contrib.auth',\n\
    'django.contrib.contenttypes',\n\
    'django.contrib.sessions',\n\
    'django.contrib.messages',\n\
    'django.contrib.staticfiles',\n\
]"

newlist = "INSTALLED_APPS = [\n\
    'django.contrib.admin',\n\
    'django.contrib.auth',\n\
    'django.contrib.contenttypes',\n\
    'django.contrib.sessions',\n\
    'django.contrib.messages',\n\
    'django.contrib.staticfiles',\n\
    'django_extensions',\n\
    'rest_framework',\n\
    'django_filters',\n\
    'simple_history',\n\
    'bookstore',\n\
]"

In [5]:
file = '/bookstore/project/settings.py'
    
with open(file, 'r') as settings_file:
    data = settings_file.read()
    data = data.replace(applist, newlist)
with open(file, 'w') as settings_file:
    write_file = settings_file.write(data)

In [6]:
middleware = "MIDDLEWARE = [\n\
    'django.middleware.security.SecurityMiddleware',\n\
    'django.contrib.sessions.middleware.SessionMiddleware',\n\
    'django.middleware.common.CommonMiddleware',\n\
    'django.middleware.csrf.CsrfViewMiddleware',\n\
    'django.contrib.auth.middleware.AuthenticationMiddleware',\n\
    'django.contrib.messages.middleware.MessageMiddleware',\n\
    'django.middleware.clickjacking.XFrameOptionsMiddleware',\n\
]"

middleware_new = "MIDDLEWARE = [\n\
    'django.middleware.security.SecurityMiddleware',\n\
    'django.contrib.sessions.middleware.SessionMiddleware',\n\
    'django.middleware.common.CommonMiddleware',\n\
    'django.middleware.csrf.CsrfViewMiddleware',\n\
    'django.contrib.auth.middleware.AuthenticationMiddleware',\n\
    'django.contrib.messages.middleware.MessageMiddleware',\n\
    'django.middleware.clickjacking.XFrameOptionsMiddleware',\n\
    'simple_history.middleware.HistoryRequestMiddleware'\n\
]"

In [7]:
file = '/bookstore/project/settings.py'
    
with open(file, 'r') as settings_file:
    data = settings_file.read()
    data = data.replace(middleware, middleware_new)
with open(file, 'w') as settings_file:
    write_file = settings_file.write(data)

## Jinja example

Web frameworks like Flask and Django, or automation ones like Ansible and Salt, provide out of the box support for Jinja. It's a natural choice for templating engine when using any of those. Ansible even uses a lot of the Jinja syntax in its Playbooks.

Jinja2 essentially needs two source ingredients, template and data that will be used to render the final document.

![title](images/jinja.png)

In [8]:
name = 'Peter'
age = 34

template = Template("My name is {{ name }} and I am {{ age }}")
msg = template.render(name=name, age=age)

print(msg)

My name is Peter and I am 34


## Getting data in and out of Django

In [9]:
file = '/bookstore/bookstore/models.py'    
    
with open(file, 'w') as models_file:
    template = Template(open('/templates/models.j2', 'r').read())
#     print(template.render(splunk=False))
    print(template.render(splunk=False))
    models_file.write(template.render())

from django.db import models
from django.conf import settings
from django.db.models.signals import post_save
from django.dispatch import receiver
import uuid
from simple_history.models import HistoricalRecords



class Author(models.Model):
    name = models.CharField("Author", max_length=20)
    history = HistoricalRecords()

    def __str__(self):
        return self.name

    class Meta:
        ordering = ('id',)


class Book(models.Model):
    title = models.CharField("Title",max_length=100)
    author = models.ForeignKey(Author,related_name='books',on_delete=models.CASCADE)
    history = HistoricalRecords()

    def __str__(self):
        return self.title

    class Meta:
        ordering = ('id',)



In [10]:
# file = '/bookstore/bookstore/signals.py'    
    
# with open(file, 'w') as file:
#     template = Template(open('/templates/signals.j2', 'r').read())
#     print(template.render())
#     file.write(template.render())

In [11]:
%%bash

cd /bookstore
python manage.py makemigrations bookstore
python manage.py migrate
python manage.py populate_history --auto
python manage.py check

Migrations for 'bookstore':


  bookstore/migrations/0001_initial.py


    - Create model Author


    - Create model HistoricalBook


    - Create model HistoricalAuthor


...

Finished saving historical records for <class 'bookstore.models.Book'>


Saving historical records for <class 'bookstore.models.Author'>


Starting bulk creating history models for <class 'bookstore.models.Author'> instances 0-200


Finished saving historical records for <class 'bookstore.models.Author'>


System check identified no issues (0 silenced).


## Update Admin

In [12]:
file = '/bookstore/bookstore/admin.py'  
    
with open(file, 'w') as admin_file:
    template = Template(open('/templates/admin.j2', 'r').read())
    print(template.render())
    admin_file.write(template.render())

from django.contrib import admin
from simple_history.admin import SimpleHistoryAdmin
from .models import *


admin.site.register(Author, SimpleHistoryAdmin)
admin.site.register(Book, SimpleHistoryAdmin)


In [13]:
%%bash

cd /bookstore
python manage.py createsuperuser

Superuser creation skipped due to not running in a TTY. You can run `manage.py createsuperuser` in your project to create one manually.


In [14]:
from django.contrib.auth.models import User
for name in ['Alice Appleseed', 'Bob Bananarama', 'Carlos Carrot']:
    user = User.objects.create_user(username=f"{name.replace(' ', '.').lower()}",
                                    email=f"{name.replace(' ', '.').lower()}@example.com",
                                    password='secret123')
    user.first_name = name.split(' ')[0]
    user.last_name = name.split(' ')[1]
    user.is_staff = True
    user.is_superuser = True
    user.save()

In [15]:
User.objects.all()

<QuerySet [<User: alice.appleseed>, <User: bob.bananarama>, <User: carlos.carrot>]>

## Challenge #1
### Explain how to login and manually create a book entry through the admin page